INSTALL
- FAISS - 
- Sent2Vec

In [1]:
import numpy as np
import faiss   

In [2]:
import os
import time
name = "Faiss-EM-OPUS-en-vi"+ str(time.strftime("%Y%m%d-%H%M"))

!mkdir $name
path = os.getcwd() + "/" + name
os.chdir(path)

!pwd
os.chdir(path)

/home/chinh/clc_data/Script/Faiss-EM-OPUS-en-vi20210304-1124


In [3]:
!wget https://github.com/hoangtrungchinh/clc_data/raw/master/opus-100-corpus.zip
!mkdir data_bin
!unzip 'opus-100-corpus.zip' -d 'data_bin'

--2021-03-04 11:24:16--  https://github.com/hoangtrungchinh/clc_data/raw/master/opus-100-corpus.zip
Resolving github.com (github.com)... 52.74.223.119
Connecting to github.com (github.com)|52.74.223.119|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/opus-100-corpus.zip [following]
--2021-03-04 11:24:17--  https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/opus-100-corpus.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30690851 (29M) [application/zip]
Saving to: ‘opus-100-corpus.zip’

opus-100-corpus.zip 100%[===================>]  29,27M  8,41MB/s    in 3,7s    

2021-03-04 11:24:23 (7,96 MB/s) - ‘opus-100-corpus.zip’ saved [30690851/30690851]

In [4]:
# import file
import numpy as np
import glob

en_files = sorted(glob.glob("data_bin/*.en"))
vi_files = sorted(glob.glob("data_bin/*.vi"))
print(en_files)
print(vi_files)

lst_en = []
lst_vi = []

start = time.time()

for file in en_files:
  with open(file) as file_in:
    for line in file_in:
      lst_en.append(line.strip())

for file in vi_files:
  with open(file) as file_in:
    for line in file_in:
      lst_vi.append(line.strip())

print(len(lst_en))
print(len(lst_vi))

['data_bin/opus.en-vi-dev.en', 'data_bin/opus.en-vi-test.en', 'data_bin/opus.en-vi-train.en']
['data_bin/opus.en-vi-dev.vi', 'data_bin/opus.en-vi-test.vi', 'data_bin/opus.en-vi-train.vi']
1004000
1004000


In [5]:
# SPLIT TRAIN, TEST, VALID
from sklearn.model_selection import train_test_split
en_train, en_test_valid, vi_train, vi_test_valid =  train_test_split(lst_en, lst_vi, test_size=0.2, random_state=123)
en_valid, en_test, vi_valid, vi_test = train_test_split(en_test_valid, vi_test_valid, test_size=0.5, random_state=123)
print(len(en_train))
print(len(vi_train))
print(len(en_valid))
print(len(vi_valid))
print(len(en_test))
print(len(vi_test))
print(en_train[20])
print(vi_train[20])
print(en_valid[-1])
print(vi_valid[-1])
print(en_test[-1])
print(vi_test[-1])

803200
803200
100400
100400
100400
100400
Childs?
Childs?
It's powered up now, isn't it?
Bây giờ nó bật rồi phải không nhỉ?
Why the hell are you going to Cuddy's baby naming?
Bị làm sao mà anh lại đi dự lễ đặt tên em bé của Cuddy chứ?


In [6]:
file_variable = [en_train,vi_train,en_valid,vi_valid,en_test,vi_test]
file_names = ["en_train","vi_train","en_valid","vi_valid","en_test","vi_test"]

for i in range(len(file_names)):
  with open(file_names[i], 'w') as f:
    for item in file_variable[i]:
      f.write(item)
      f.write("\n")
    
!ls -al

total 105172
drwxrwxr-x 3 chinh chinh     4096 Thg 3   4 11:24 .
drwxrwxr-x 5 chinh chinh     4096 Thg 3   4 11:24 ..
drwxrwxr-x 2 chinh chinh     4096 Thg 3   4 11:24 data_bin
-rw-rw-r-- 1 chinh chinh  3318349 Thg 3   4 11:24 en_test
-rw-rw-r-- 1 chinh chinh 26563375 Thg 3   4 11:24 en_train
-rw-rw-r-- 1 chinh chinh  3328557 Thg 3   4 11:24 en_valid
-rw-rw-r-- 1 chinh chinh 30690851 Thg 3   4 11:24 opus-100-corpus.zip
-rw-rw-r-- 1 chinh chinh  4365722 Thg 3   4 11:24 vi_test
-rw-rw-r-- 1 chinh chinh 35019161 Thg 3   4 11:24 vi_train
-rw-rw-r-- 1 chinh chinh  4382771 Thg 3   4 11:24 vi_valid


In [7]:
start = time.time()
import sent2vec
model = sent2vec.Sent2vecModel()
model.load_model('../../../wiki_unigrams.bin')

print('=== Ending, Total time (second): ', time.time() - start)

=== Ending, Total time (second):  2.2136025428771973


In [8]:
start = time.time()
srcfile = "en_train"
tarfile = "vi_train"
lst_srcfile = en_train
lst_tarfile = vi_train

vec_lst_srcfile = model.embed_sentences(lst_srcfile)
print('=== Ending, Total time (second): ', time.time() - start)

Test Cosin similarity

In [9]:
import faiss

start = time.time()
index = faiss.IndexFlatL2(vec_lst_srcfile.shape[1])
index.ntotal
faiss.normalize_L2(vec_lst_srcfile)
index.add(vec_lst_srcfile)
k = 6
distance, index = index.search(vec_lst_srcfile, k)

print('=== Ending, Total time (second): ', time.time() - start)
print(len(distance))
print(len(index))

print(index)
print(distance)

=== Ending, Total time (second):  2348.1285033226013
803200
803200
[[ 27848  76309  82411 133450   1599      0]
 [     1  99162 354835 267111  59188  44528]
 [135971 150398 239119 270481  63366  62419]
 ...
 [562684  66033  68748  53991  76993  45541]
 [ 43211 180813 296147 300498 292964 145745]
 [153425 151219 803199 244144 112858 597122]]
[[7.1525574e-07 7.1525574e-07 7.1525574e-07 7.1525574e-07 7.1525574e-07
  7.1525574e-07]
 [1.1920929e-07 3.1163800e-01 3.7609673e-01 3.9664578e-01 4.7298753e-01
  5.5491567e-01]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00]
 ...
 [1.1920929e-07 2.3841858e-07 2.3841858e-07 2.3841858e-07 2.3841858e-07
  2.3841858e-07]
 [4.7683716e-07 4.7683716e-07 4.7683716e-07 4.7683716e-07 4.7683716e-07
  4.7683716e-07]
 [2.3841858e-07 2.3841858e-07 2.3841858e-07 2.3841858e-07 2.3841858e-07
  8.9151144e-02]]


In [25]:
j = k-1
# TEST
for i in range(10):
    a = vec_lst_srcfile[i]
    b = vec_lst_srcfile[index[i][j]]
    dist_squared = np.sum(np.square(a - b))
    print("index:",index[i][j], "distance:",distance[i][j], "tested:",dist_squared)

index: 0 distance: 7.1525574e-07 tested: 0.0
index: 44528 distance: 0.55491567 tested: 0.5549159
index: 62419 distance: 0.0 tested: 2.2448846e-15
index: 20 distance: 0.0 tested: 0.0
index: 326281 distance: 1.1920929e-07 tested: 0.0
index: 726707 distance: 0.06721616 tested: 0.067215934
index: 10938 distance: 0.0 tested: 0.0
index: 171072 distance: 0.3191253 tested: 0.31912494
index: 749825 distance: 0.22028756 tested: 0.22028747
index: 582649 distance: 0.07324958 tested: 0.0732492


In [12]:
start = time.time()
saperate = " || "

src_label = "S"
emb_label = "E"
thres_arr = [0.9, 0.8, 0.7, 0.6]
for i in range(len(thres_arr)):
    threshold = thres_arr[i]
    out_file_write = open("en_train_EM_"+str(thres_arr[i]), 'w')
    out_file_write_score = open("en_train_EM_score_"+str(thres_arr[i]), 'w')
    out_file_write_factor = open("en_train_EM_factor_"+str(thres_arr[i]), 'w')
    num_gth_thres = 0

    for i in range(len(lst_srcfile)):
        score = distance[i][1]
        content = ""
        content_label_arr = []
        if score >= threshold:
            num_gth_thres += 1
            best_simi_index = index[i][1]
            content = lst_srcfile[i] + saperate + lst_tarfile[best_simi_index]

            content_label = [src_label] * len(lst_srcfile[i].split())
            content_label.append(emb_label)
            content_label = content_label + [emb_label] * len(lst_tarfile[best_simi_index].split())

            content_label_arr = " ".join(content_label)
        else:
            content = lst_srcfile[i]
            content_label = [src_label] * len(lst_srcfile[i].split())
            content_label_arr = " ".join(content_label)

        out_file_write.writelines(content+ "\n")    
        out_file_write_factor.writelines(content_label_arr + "\n")
        out_file_write_score.writelines(str(score) + "\n")

    print('threshold =', threshold, "EM sentences =", num_gth_thres, "/", len(lst_srcfile), num_gth_thres/len(lst_srcfile), "%" )

    out_file_write.close()
    out_file_write_score.close()
    out_file_write_factor.close()

done = time.time()
elapsed = done - start
print('=== Ending, Total time (second): ', elapsed)
    


threshold = 0.9 EM sentences = 19697 / 803200 0.02452315737051793 %
threshold = 0.8 EM sentences = 43414 / 803200 0.05405129482071713 %
threshold = 0.7 EM sentences = 77682 / 803200 0.0967156374501992 %
threshold = 0.6 EM sentences = 119353 / 803200 0.14859686254980078 %
=== Ending, Total time (second):  21.564908027648926
